In [2]:
import networkx as nx
from collections import deque

In [3]:
def find_augmentpath(N, s, t):
    P = [-1] * nx.number_of_nodes(N)
    visited = set()
    stack = deque()
    stack.appendleft(s)
    while stack:
        v = stack.popleft()
        if v == t:
            return P, True
        if not v in visited:
            visited.add(v)
            for w in N.neighbors(v):
                if not w in visited and N.edges[v, w]['weight'] > 0:
                    stack.appendleft(w)
                    P[w] = v
    return P, False

def restore_shortestpath(u, v, P):
    path = []
    temp = v
    while temp != u:
        parent = P[temp]
        path.append((parent, temp))
        temp = parent
    path.reverse()
    return path


def min_capacity(N, path):
    min_cap = float('inf')
    for u, v in path:
        capacity = N.edges[u, v]['weight']
        if capacity < min_cap:
            min_cap = capacity
    return min_cap

def increase_flow(N, path, amount, flow):
    for u, v in path:
        if flow[(v, u)] <= 0:
            flow[(u, v)] += amount
        else:
            diff = flow[(v, u)] - amount
            if diff >= 0:
                flow[(v, u)] = diff
            else:
                flow[(u, v)] = -diff
                flow[(v, u)] = 0
        N.edges[u, v]['weight'] -= amount
        if N.has_edge(v, u):
            N.edges[v, u]['weight'] += amount
        else:
            N.add_edge(v, u, weight=amount)


def my_Ford_Fulkerson(G, s, t):
    N = G.copy()
    f = {}
    for u, v in N.edges:
        f[(u, v)] = 0
        f[(v, u)] = 0
    P, is_found = find_augmentpath(N, s, t)
    while is_found:
        augmentpath = restore_shortestpath(s, t, P)
        min_cap = min_capacity(N, augmentpath)
        increase_flow(N, augmentpath, min_cap, f)
        P, is_found = find_augmentpath(N, s, t)
    return N, f


In [4]:
G = nx.read_weighted_edgelist("11-2-1assign.edgelist", create_using=nx.DiGraph(), nodetype=int)

N, f = my_Ford_Fulkerson(G, 0, 5)

def mincut_dfs_old(N,s):
    #到達済みをあらかじめ作成
    visited = set()
    stack = deque()
    stack.appendleft(s)
    #スタックから順番に取り出して到達済みにいれる
    while stack:
        v  = stack.popleft()
        if not v in visited:
            visited.add(v)
            #それぞれに到達済みでなく容量0以上ならスタックに入れる
            for w in N.neighbors(v):
                if not w in visited and N.get_edge_data(v, w)['weight'] > 0:
                    stack.append(w)
    #全体から用済みと最初の点ををB,到達済みをAとして返す。
    cut = set(N.nodes()) - visited
    A = visited
    B = cut - {s}
    
    return A, B

def mincut_dfs_new(N, s):
    visited = set()
    stack = deque()
    stack.appendleft(s)
    while stack:
        v = stack.popleft()
        if not v in visited:
            visited.add(v)
            for w in N.neighbors(v):
                if not w in visited and N.edges[v, w]['weight'] > 0:
                    stack.appendleft(w)
    return visited, set(N.nodes)-visited


A, B = mincut_dfs_new(N,0)
print("A = {0}, B = {1}".format(A, B))

A = {0, 1, 2, 3, 6, 7, 8, 9, 10, 11, 12}, B = {13, 4, 5}
